In [1]:
from operator import imod
import sys
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import LearningRateScheduler, LambdaCallback
from tensorflow.keras.models import load_model
from pathlib import Path
import platform
sys.path.append(os.path.abspath("../../src"))
from dual_network import dual_network, DN_INPUT_SHAPE
from auto_play import AutoPlay
from const import HISTORY_DIR, MODEL_DIR
from train_network import train_network, update_best_player, load_data


In [1]:
from auto_play_case3 import AutoPlayCase3
current_dir = %pwd
best_model = current_dir+"/case3_best.h5"
latest_model = current_dir+"/case3_latest.h5"
history_path = current_dir*"/ex_data_input_shape.history"
dual_network(best_model, (40, 5, 2))


train_network(best_model, latest_model, history_path=history_path, shape=(40, 5, 2))

eval_best_player = AutoPlayCase3()
eval_best_player.action1("pv_ismcts", latest_model)
eval_best_player.action2("ismcts")
eval_best_player.calc_win_rate(5, 100, 0.0, 'Evaluate Best Player vs ismcts', True)



PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True
Train 0 ====================
Evaluate New Network vs Old
Completed 100/100Win rate : 0.74
Change BestPlayer
Evaluate Best Player vs ismcts
Completed 100/100Win rate : 0.08000000000000002
Train 1 ====================
Self Play
Completed 36/2000

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True
Train 0 ====================
Self Play
Completed 2000/2000PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True
Train 100/100
Evaluate New Network vs Old
Completed 100/100Win rate : 0.66
Change BestPlayer
Evaluate Best Player vs ismcts
Completed 100/100Win rate : 0.08000000000000002
Train 1 ====================
Self Play
Completed 8/2000

Train 4 ====================
Self Play
Completed 2000/2000PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True
Train 100/100
Evaluate New Network vs Old
Completed 100/100Win rate : 0.46


PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True


NameError: name 'i' is not defined